In [ ]:
# stance_summary_module.py

import os
import torch
import torch.nn as nn
import joblib
import numpy as np
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


class StanceModel:
    def __init__(self, input_dim=4, output_dim=3, model_dir="D:/NLP/saved_models"):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model_dir = model_dir
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.tfidf = None
        self.label_encoder = None

    def load(self):
        """Load the trained model, TF-IDF vectorizer, and label encoder."""
        # Load model
        self.model = nn.Sequential(
            nn.Linear(self.input_dim + 5000, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, self.output_dim),
            nn.Softmax(dim=1)
        ).to(self.device)

        self.model.load_state_dict(torch.load(
            os.path.join(self.model_dir, "stance_model.pt"),
            map_location=self.device
        ))

        # Load vectorizer and label encoder
        self.tfidf = joblib.load(os.path.join(self.model_dir, "tfidf_vectorizer.joblib"))
        self.label_encoder = joblib.load(os.path.join(self.model_dir, "label_encoder.joblib"))
        self.model.eval()

    def save(self):
        """Save the model weights and other components to disk."""
        torch.save(self.model.state_dict(), os.path.join(self.model_dir, "stance_model.pt"))
        joblib.dump(self.tfidf, os.path.join(self.model_dir, "tfidf_vectorizer.joblib"))
        joblib.dump(self.label_encoder, os.path.join(self.model_dir, "label_encoder.joblib"))

    def predict(self, features):
        """Predict stance based on input features."""
        with torch.no_grad():
            return self.model(features)


class StanceSummaryModel:
    def __init__(self, stance_model_dir, summarizer_model="facebook/bart-large-cnn"):
        self.stance_model = StanceModel(input_dim=4, output_dim=3, model_dir=stance_model_dir)
        self.stance_model.load()
        self.analyzer = SentimentIntensityAnalyzer()
        self.summarizer = pipeline("summarization", model=summarizer_model)

    def summarize_and_analyze(self, text):
        """Summarize the text and perform stance and sentiment analysis."""
        # Step 1: Summarize the input text
        summary = self.summarizer(text, max_length=150, min_length=50, do_sample=False)[0]['summary_text']

        # Step 2: Sentiment features using VADER
        sentiment_scores = self.analyzer.polarity_scores(summary)
        sentiment_features = np.array([
            sentiment_scores['neg'],
            sentiment_scores['neu'],
            sentiment_scores['pos'],
            sentiment_scores['compound']
        ]).reshape(1, -1)

        # Step 3: TF-IDF + Sentiment
        tfidf_features = self.stance_model.tfidf.transform([summary]).toarray()
        combined_features = np.hstack([tfidf_features, sentiment_features])

        # Step 4: Predict stance
        input_tensor = torch.tensor(combined_features, dtype=torch.float32).to(self.stance_model.device)
        output = self.stance_model.predict(input_tensor)
        stance_index = torch.argmax(output, dim=1).item()

        # Step 5: Bias score (compound sentiment)
        bias_score = sentiment_scores['compound']

        return summary, stance_index, bias_score


def run_stance_analysis(model_dir, text_file_path):
    """Load text from file, run stance and sentiment analysis, and return the results."""
    if not os.path.exists(text_file_path):
        raise FileNotFoundError(f"❌ File not found: {text_file_path}")

    with open(text_file_path, "r", encoding="utf-8") as file:
        input_text = file.read()

    stance_summary_model = StanceSummaryModel(stance_model_dir=model_dir)
    summary, stance, bias_score = stance_summary_model.summarize_and_analyze(input_text)

    stance_labels = ["Political", "Terrorist", "Freedom Fighter"]
    stance_label = stance_labels[stance] if stance < len(stance_labels) else "Unknown"

    return {
        "summary": summary,
        "predicted_stance": stance_label,
        "bias_score": bias_score
    }


if __name__ == "__main__":
    model_path = "D:/NLP/saved_models"
    text_path = "D:/NLP/221501184/processed_model.txt"
    
    try:
        results = run_stance_analysis(model_path, text_path)
        print("Summary:", results["summary"])
        print("Predicted Stance:", results["predicted_stance"])
        print("Bias Score:", results["bias_score"])
    except Exception as e:
        print(str(e))


Device set to use cuda:0


Summary: Michelle Béghin is a volunteer in a humanitarian mission in one of our first if of Au.AUROVI university structure - Bites -Partners in South America. She is also active with PartnersLocaux Gnelefrançais and Lallemandàdes Party works in the experienced city like the Down All Else.
Predicted Stance: Freedom Fighter
Bias Score: 0.7845
